# Montar Drive + Imports

In [1]:
import os
from google.colab import drive
from typing import Dict, Any, List, Union
import pandas as pd
from tqdm import tqdm
import numpy as np
import networkx as nx
import warnings
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import shapiro, levene, f_oneway, ttest_ind, mannwhitneyu, kruskal
from itertools import combinations

In [2]:
# 1. Montar Google Drive
print("Paso 1: Montando Google Drive...")
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/MyDrive/Máster - Data Science/TFM/Data/ESTUDIO_MULTIPLEX')

Paso 1: Montando Google Drive...
Mounted at /content/drive


# Función de recuperación de información + listar pacientes

In [3]:
def obtener_datos_paciente(
    patient_id: str,
    base_dir: str = 'DADES_CORRECTED'
) -> Union[Dict[str, Any], None]:
    """
    Recupera toda la información (clínica, nodos y redes) para un paciente
    dado su ID, a partir de la estructura de archivos en base_dir.

    Args:
        patient_id: El ID del paciente (ej. 'P001').
        base_dir: El directorio base donde se encuentran los archivos.

    Returns:
        Un diccionario con la información del paciente en el formato solicitado,
        o None si el ID del paciente no se encuentra.
    """

    # Rutas a los archivos
    clinic_file = os.path.join(base_dir, 'CLINIC.csv')
    nodes_vol_file = os.path.join(base_dir, 'NODES.csv')
    nodes_fa_file = os.path.join(base_dir, 'GM_FA.csv')
    nodes_md_file = os.path.join(base_dir, 'GM_MD.csv')

    # 1. Recuperar información clínica
    try:
        df_clinic = pd.read_csv(clinic_file)

        # Buscar la fila del paciente por su ID
        patient_data = df_clinic[df_clinic['ids'] == patient_id]

        if patient_data.empty:
            print(f"Error: ID de paciente '{patient_id}' no encontrado en CLINIC.csv.")
            return None

        # Extraer los datos (la fila es un DataFrame de 1xN, usamos .iloc[0] para obtener la Serie)
        patient_series = patient_data.iloc[0]

    except FileNotFoundError:
        print(f"Error: Archivo CLINIC.csv no encontrado en {base_dir}")
        return None

    # 2. Mapear y codificar la información clínica al formato solicitado

    # El campo 'origenes' se usa para el campo 'origen' principal
    origen_data = str(patient_series['origenes'])

    # El campo 'sites' se mapea a 'site' en la respuesta
    site_data = str(patient_series['sites'])

    # Asumimos que los campos 'sexes', 'controls_mses' y 'mstypes' están
    # codificados como enteros o pueden ser convertidos a float/int.
    # Necesitarás definir la codificación de 'sexes' y 'mstypes' para el formato final,
    # aquí se usarán los valores tal cual están en el CSV (o se fuerza a int/float).

    # Nota: Los nombres de las columnas ddes y edsses en el CSV anterior eran ddes y edsses
    # pero el formato solicitado pide 'dd' y 'edss'.
    info_clinica = {
        'id': str(patient_series['ids']),
        'age': float(patient_series['ages']),
        # Se asume una codificación para 'sexes', aquí se devuelve como está
        'sex': int(patient_series['sexes']) if pd.notna(patient_series['sexes']) else np.nan,
        'dd': float(patient_series['ddes']),
        'edss': float(patient_series['edsses']),
        'control_ms': int(patient_series['controls_mses']),
        # Se asume una codificación para 'mstypes' (0=RRMS, 1=SPMS, 2=PPMS)
        'mstype': int(patient_series['mstypes']) if pd.notna(patient_series['mstypes']) else np.nan
    }

    # 3. Recuperar datos de nodos (nodos_volumetrico, nodos_FA, nodos_MD)

    def load_node_data(file_path: str) -> List[float]:
        """Carga y devuelve la lista de valores de nodo para el paciente."""
        try:
            df_nodes = pd.read_csv(file_path)
            patient_row = df_nodes[df_nodes['ID'] == patient_id]
            if not patient_row.empty:
                # Excluir la columna 'ID' y convertir los valores a lista (float)
                return patient_row.iloc[0, 1:].tolist()
            else:
                return []
        except FileNotFoundError:
            print(f"Advertencia: Archivo de nodos no encontrado: {file_path}")
            return []

    nodos_volumetrico = load_node_data(nodes_vol_file)
    nodos_FA = load_node_data(nodes_fa_file)
    nodos_MD = load_node_data(nodes_md_file)

    # 4. Recuperar matrices de conectividad (redes)

    redes = {}
    network_types = {'FA': 'FA_network', 'GM': 'GM_network', 'rsfmri': 'rsfmri_network'}

    for net_key, net_dir in network_types.items():
        network_path = os.path.join(base_dir, net_dir, f"{patient_id}.csv")
        try:
            # Leer la matriz sin encabezados ni índice
            matrix = pd.read_csv(network_path, header=None, index_col=None).values
            redes[net_key] = matrix
        except FileNotFoundError:
            print(f"Advertencia: Matriz {net_key} no encontrada para {patient_id} en {network_path}")
            # Si una matriz falta, se puede optar por devolver un array vacío o None
            redes[net_key] = np.array([])

    # 5. Consolidar el resultado

    resultado = {
        'origen': origen_data,
        'site': site_data, # Incluir 'site' ya que está en los datos clínicos
        'info_clinica': info_clinica,
        'nodos_volumetrico': nodos_volumetrico,
        'nodos_FA': nodos_FA,
        'nodos_MD': nodos_MD,
        'redes': redes
    }

    return resultado

In [4]:
def listar_pacientes(base_dir: str = ".") -> list[str]:
    """
    Devuelve una lista con todos los valores de 'id_paciente' del archivo patients.csv.

    Args:
        base_dir (str): Carpeta base donde se encuentra patients.csv

    Returns:
        list[str]: Lista de IDs de pacientes.
    """
    patients_path = os.path.join(base_dir, "patients.csv")

    if not os.path.exists(patients_path):
        raise FileNotFoundError(f"No se encontró el archivo: {patients_path}")

    df_patients = pd.read_csv(patients_path)

    if "id_paciente" not in df_patients.columns:
        raise ValueError("El archivo patients.csv no contiene la columna 'id_paciente'.")

    # Elimina valores nulos y los convierte a string
    ids = df_patients["id_paciente"].dropna().astype(str).tolist()

    return ids

pacientes = listar_pacientes()

# Cargar los datos

Obtenemos los datos en formato tensorial para procesarlos luego:

- `FA_matrices_tensor[i][j]`, `GM_matrices_tensor[i][j]` y `rsfMRI_matrices_tensor[i][j]` → lista de longitud `n_pacientes` con todos los valores de la conexión `(i,j)` para todos los pacientes.

- `nodos_volumetricos_tensor[k]`, `nodos_FA_tensor[k]` y `nodos_MD_tensor[k]` → listas de valores por nodo `k` para todos los pacientes.

- `origen`, `ages`, `sexes`, `ddes`, `edsses`, `control_mses`, `mstypes`  → arrays con las variables clínicas.

- `site`  → array con el protocolo de muestra (FIS, MSVIS o sub)

In [5]:
# --- Obtener dimensiones base (suponiendo todos los pacientes tienen mismas dimensiones) ---
info_ref = obtener_datos_paciente(pacientes[0])
n_nodos = info_ref['redes']['FA'].shape[0]

# --- Inicializar estructuras vacías ---
FA_matrices_tensor = [[[] for _ in range(n_nodos)] for _ in range(n_nodos)]
GM_matrices_tensor = [[[] for _ in range(n_nodos)] for _ in range(n_nodos)]
rsfMRI_matrices_tensor = [[[] for _ in range(n_nodos)] for _ in range(n_nodos)]

nodos_volumetricos_tensor = [[] for _ in range(n_nodos)]
nodos_FA_tensor = [[] for _ in range(n_nodos)]
nodos_MD_tensor = [[] for _ in range(n_nodos)]

# Variables clínicas
origenes = []
ids = []
sites = []
ages = []
sexes = []
ddes = []
edsses = []
controls_mses = []
mstypes = []

# --- Iterar sobre pacientes ---
for paciente in tqdm(pacientes, desc="Construyendo tensores de conectividad"):
    info = obtener_datos_paciente(paciente)
    redes = info['redes']
    clinica = info['info_clinica']

    # Guardar variables clínicas
    origenes.append(info['origen'])
    ids.append(clinica['id'])
    sites.append(info['site'])
    ages.append(clinica['age'])
    sexes.append(clinica['sex'])
    controls_mses.append(clinica['control_ms'])
    ddes.append(clinica['dd'])
    edsses.append(clinica['edss'])
    mstypes.append(clinica['mstype'])

    # --- Rellenar tensores de conectividad ---
    for i in range(n_nodos):
        for j in range(n_nodos):
            FA_matrices_tensor[i][j].append(redes['FA'][i, j])
            GM_matrices_tensor[i][j].append(redes['GM'][i, j])
            rsfMRI_matrices_tensor[i][j].append(redes['rsfmri'][i, j])

    # --- Rellenar tensores de nodos ---
    for k in range(n_nodos):
        nodos_volumetricos_tensor[k].append(info['nodos_volumetrico'][k])
        nodos_FA_tensor[k].append(info['nodos_FA'][k])
        nodos_MD_tensor[k].append(info['nodos_MD'][k])


# Convertir a numpy
FA_matrices_tensor = np.array(FA_matrices_tensor) # Dimensión: (n_nodos, n_nodos, n_pacientes)
GM_matrices_tensor = np.array(GM_matrices_tensor)
rsfMRI_matrices_tensor = np.array(rsfMRI_matrices_tensor)

nodos_volumetricos_tensor = np.array(nodos_volumetricos_tensor)
nodos_FA_tensor = np.array(nodos_FA_tensor)
nodos_MD_tensor = np.array(nodos_MD_tensor)


# Convertir edad y sexo a arrays NumPy
ages = np.array(ages)
sexes = np.array(sexes)
ddes = np.array(ddes)
edsses = np.array(edsses)
controls_mses = np.array(controls_mses)
mstypes = np.array(mstypes)

Construyendo tensores de conectividad: 100%|██████████| 270/270 [04:10<00:00,  1.08it/s]


# Guardar multilayer

In [6]:
def combinar_tensores(t1, t2, t3):
    """
    t1, t2, t3: tensores de tamaño (n_nodos, n_nodos, n_pacientes)

    Devuelve:
        tensor_out de tamaño (2*n_nodos, 2*n_nodos, n_pacientes)
        tal que para cada paciente p:

            ( t1[:,:,p]   t2[:,:,p] )
            ( t2[:,:,p]   t3[:,:,p] )
    """

    n_nodos, _, n_pacientes = t1.shape

    # Tensor de salida
    out = np.zeros((2*n_nodos, 2*n_nodos, n_pacientes))

    # Rellenar bloques
    out[:n_nodos, :n_nodos, :] = t1
    out[:n_nodos, n_nodos:, :] = t2
    out[n_nodos:, :n_nodos, :] = t2
    out[n_nodos:, n_nodos:, :] = t3

    return out

In [7]:
multilayer_matrices_tensor = combinar_tensores(GM_matrices_tensor, FA_matrices_tensor, rsfMRI_matrices_tensor)

In [8]:
def guardar_matrices_pacientes_csv(tensor, pacientes, base_dir="DADES_CORRECTED/multi_network"):
    """
    Guarda cada matriz (2*n_nodos, 2*n_nodos) correspondiente a cada paciente en formato CSV.

    Parameters
    ----------
    tensor : np.ndarray
        Tensor de tamaño (2*n_nodos, 2*n_nodos, n_pacientes).
    pacientes : list
        Lista con los nombres de cada paciente.
    base_dir : str
        Carpeta donde se guardarán las matrices CSV.
    """

    # Crear carpeta si no existe
    os.makedirs(base_dir, exist_ok=True)

    # Verificar consistencia
    if tensor.shape[2] != len(pacientes):
        raise ValueError("El número de pacientes no coincide con la tercera dimensión del tensor.")

    # Guardar cada matriz como CSV
    for i, nombre in enumerate(pacientes):
        matriz = tensor[:, :, i]
        ruta_archivo = os.path.join(base_dir, f"{nombre}.csv")
        np.savetxt(ruta_archivo, matriz, delimiter=",")

    print("CSV guardados correctamente.")


In [9]:
guardar_matrices_pacientes_csv(multilayer_matrices_tensor, pacientes, base_dir="DADES_CORRECTED/multi_network")

CSV guardados correctamente.
